In [1]:
## RESOURCES
# https://docs.microsoft.com/de-de/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py
# https://docs.microsoft.com/de-de/python/api/azureml-core/?view=azure-ml-py
# https://github.com/microsoft/MLHyperparameterTuning
# https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-train-scikit-learn.md
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py
# https://docs.microsoft.com/de-de/azure/machine-learning/how-to-tune-hyperparameters
# https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-configure-auto-train.md


In [2]:
from azureml.core import Workspace, Experiment

In [3]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-193282
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-193282


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RBGJL9Y5B to authenticate.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# https://docs.microsoft.com/de-de/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

cluster_name = "Udacity-AML-Cluster"

config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=3)
comp_target = ComputeTarget.create(ws, cluster_name, config)
comp_target.wait_for_completion(show_output=True)

Create cluster
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint
import os

# Specify parameter sampler
# https://docs.microsoft.com/de-de/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py

ps = RandomParameterSampling({
    "C" : uniform(0.5, 2),
    "max_iter": randint(100)
})

# Specify a Policy
# https://docs.microsoft.com/de-de/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

policy = BanditPolicy(slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py
# https://github.com/microsoft/MLHyperparameterTuning/blob/master/04_Hyperparameter_Random_Search.ipynb

est = SKLearn(
    compute_target=comp_target,
    conda_packages=['pyarrow>=0.12.0'],
    source_directory='./',
    entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# https://github.com/microsoft/MLHyperparameterTuning/blob/master/04_Hyperparameter_Random_Search.ipynb

hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    estimator=est,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=2,
    max_total_runs=20
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

run = exp.submit(hyperdrive_config)

RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [32]:
# Get your best run and save the model from that run.
# https://github.com/microsoft/MLHyperparameterTuning/blob/master/04_Hyperparameter_Random_Search.ipynb

run.wait_for_completion(show_output=True)
best_run_hyperdrive = run.get_best_run_by_primary_metric()

# Save best model
# https://docs.microsoft.com/de-de/python/api/azureml-core/azureml.core.run.run?view=azure-ml-py#azureml-core-run-run-register-model

best_model_hyperdrive = best_run_hyperdrive.register_model(model_name='best_hyperdrive_model', model_path='outputs/model.joblib')


RunId: HD_ee6b48e2-1afa-4320-b484-cea49b4f2d81
Web View: https://ml.azure.com/runs/HD_ee6b48e2-1afa-4320-b484-cea49b4f2d81?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-193282/workspaces/quick-starts-ws-193282&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_ee6b48e2-1afa-4320-b484-cea49b4f2d81
Web View: https://ml.azure.com/runs/HD_ee6b48e2-1afa-4320-b484-cea49b4f2d81?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-193282/workspaces/quick-starts-ws-193282&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
# https://docs.microsoft.com/de-de/python/api/azureml-core/azureml.data.dataset_factory.tabulardatasetfactory?view=azure-ml-py

ds = TabularDatasetFactory().from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Integrate label column in dataset. 
# The function AutoMLConfig requires an object 'training_data' and a 'label_column_name' that specifies the label column in the training_data object.

training_data = x
training_data['label'] = y

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-configure-auto-train.md

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="label",
    n_cross_validations=5)

# Submit your automl run

In [12]:
from azureml.core.experiment import Experiment

# https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-configure-auto-train.md

exp_name = "Automl"
project_folder = "Automl"

exp_automl = Experiment(ws, exp_name)

run_automl = exp_automl.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Automl,AutoML_a3d0314b-9d05-43f7-bb24-44cc502aabd8,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

In [28]:
# Retrieve and save your best automl model.
# https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-configure-auto-train.md

best_run_automl = run_automl.get_best_child()

best_model_automl = best_run_automl.register_model(model_name = best_run_automl.properties['model_name'], model_path = 'outputs/model.pkl')